In [1]:
import sys
import os
import pandas as pd
import numpy as np
from pyspark.mllib.feature import StandardScaler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import NaiveBayes
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorSlicer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [2]:
spark = SparkSession.builder.config("spark.executor.pyspark.memory", "6g") \
        .config("spark.executor.memory", "6g").config("spark.driver.memory", "6g") \
        .config("spark.driver.maxResultSize", "6g").getOrCreate()

22/03/31 10:55:00 WARN Utils: Your hostname, sandeepPC resolves to a loopback address: 127.0.1.1; using 192.168.29.58 instead (on interface enp4s0)
22/03/31 10:55:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/03/31 10:55:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
select_cols = input()
if(int(select_cols)):
    df = spark.read.csv('hdfs://localhost:9000/user/sandeep/data/datasetfinal.csv', sep=',', inferSchema=True, header=True).select("Duration","Dst_bytes","Flag","Hot","Num_compromised","Count","Serror_rate","Rerror_rate","Same_srv_rate","Diff_srv_rate","Srv_count","Srv_serror_rate","Srv_rerror_rate","Srv_diff_host_rate","Dst_host_count","Dst_host_srv_count","Dst_host_same_srv_rate","Dst_host_diff_srv_rate","Dst_host_same_src_port_rate","Dst_host_srv_diff_host_rate","Dst_host_serror_rate","Dst_host_srv_serror_rate","Dst_host_rerror_rate","class","class_index")
else:
    df = spark.read.csv('hdfs://localhost:9000/user/sandeep/data/datasetfinal.csv', sep=',', inferSchema=True, header=True)

1


In [4]:
spark.sparkContext

<SparkContext master=local[*] appName=pyspark-shell>

In [5]:
df.dtypes

[('Duration', 'int'),
 ('Dst_bytes', 'int'),
 ('Flag', 'int'),
 ('Hot', 'int'),
 ('Num_compromised', 'int'),
 ('Count', 'int'),
 ('Serror_rate', 'int'),
 ('Rerror_rate', 'double'),
 ('Same_srv_rate', 'double'),
 ('Diff_srv_rate', 'double'),
 ('Srv_count', 'double'),
 ('Srv_serror_rate', 'double'),
 ('Srv_rerror_rate', 'double'),
 ('Srv_diff_host_rate', 'double'),
 ('Dst_host_count', 'int'),
 ('Dst_host_srv_count', 'int'),
 ('Dst_host_same_srv_rate', 'double'),
 ('Dst_host_diff_srv_rate', 'double'),
 ('Dst_host_same_src_port_rate', 'double'),
 ('Dst_host_srv_diff_host_rate', 'double'),
 ('Dst_host_serror_rate', 'double'),
 ('Dst_host_srv_serror_rate', 'double'),
 ('Dst_host_rerror_rate', 'double'),
 ('class', 'string'),
 ('class_index', 'double')]

In [6]:
catcols = [x for (x, dataType) in df.dtypes if dataType == "string"]
numcols = [x for (x, dataType) in df.dtypes if ((dataType =="int") | (dataType == "double") & (x != "class_index"))]

In [7]:
numcols

['Duration',
 'Dst_bytes',
 'Flag',
 'Hot',
 'Num_compromised',
 'Count',
 'Serror_rate',
 'Rerror_rate',
 'Same_srv_rate',
 'Diff_srv_rate',
 'Srv_count',
 'Srv_serror_rate',
 'Srv_rerror_rate',
 'Srv_diff_host_rate',
 'Dst_host_count',
 'Dst_host_srv_count',
 'Dst_host_same_srv_rate',
 'Dst_host_diff_srv_rate',
 'Dst_host_same_src_port_rate',
 'Dst_host_srv_diff_host_rate',
 'Dst_host_serror_rate',
 'Dst_host_srv_serror_rate',
 'Dst_host_rerror_rate']

In [8]:
from pyspark.ml.feature import StandardScaler, VectorAssembler 
vector_assembler = VectorAssembler(inputCols = numcols, outputCol = "features")
temptrain = vector_assembler.setHandleInvalid("skip").transform(df).select("class_index","features")

In [9]:
# temptrain.select("class_index","features").show(5)

In [10]:
df = StandardScaler(inputCol = "features", outputCol="scaledfeatures").fit(temptrain).transform(temptrain)

22/03/31 10:55:23 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [11]:
train, test = df.randomSplit([0.7, 0.3], seed = 2022)

In [12]:
train.select("scaledfeatures").show(5)

+--------------------+
|      scaledfeatures|
+--------------------+
|(23,[1,5,6,11,14,...|
|(23,[1,5,6,11,14,...|
|(23,[1,5,6,11,14,...|
|(23,[1,5,6,11,14,...|
|(23,[1,5,6,11,14,...|
+--------------------+
only showing top 5 rows



In [13]:
select_classifier = input()

0


In [14]:
if(int(select_classifier)):
    model = RandomForestClassifier(featuresCol = 'scaledfeatures', labelCol = 'class_index').fit(train)
else:
    model = NaiveBayes(smoothing=1.0, modelType="multinomial", featuresCol = 'scaledfeatures', labelCol = 'class_index').fit(train)

In [15]:
predictions = model.transform(test)

In [16]:
predictions.show()

+-----------+--------------------+--------------------+--------------------+--------------------+----------+
|class_index|            features|      scaledfeatures|       rawPrediction|         probability|prediction|
+-----------+--------------------+--------------------+--------------------+--------------------+----------+
|        0.0|(23,[1,5,6,11,14,...|(23,[1,5,6,11,14,...|[-16.326620385696...|[0.75557848134042...|       0.0|
|        0.0|(23,[1,5,6,11,14,...|(23,[1,5,6,11,14,...|[-20.703135113760...|[0.86732536684012...|       0.0|
|        0.0|(23,[1,5,6,11,14,...|(23,[1,5,6,11,14,...|[-22.131355367547...|[0.99997995513078...|       0.0|
|        0.0|(23,[1,5,6,11,14,...|(23,[1,5,6,11,14,...|[-22.192098027096...|[0.99998052869209...|       0.0|
|        0.0|(23,[1,5,6,11,14,...|(23,[1,5,6,11,14,...|[-30.499637352015...|[0.99999707081234...|       0.0|
|        0.0|(23,[1,5,6,11,14,...|(23,[1,5,6,11,14,...|[-30.630260650416...|[0.99999724513132...|       0.0|
|        0.0|(23,[1

In [17]:
evaluator = MulticlassClassificationEvaluator( labelCol="class_index", predictionCol="prediction", metricName="accuracy")

In [18]:
accuracy = evaluator.evaluate(predictions)
accuracy

22/03/31 10:56:06 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/03/31 10:56:06 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


0.9040231163690373

In [19]:
# rf.featureImportances

In [20]:
if(int(select_classifier)):
    figure(figsize=(12, 10), dpi=100)
    plt.barh(numcols, model.featureImportances)